<h1><center>Spark Structured Streaming</center></h1>
<hr><hr><hr>

- Stackoverflow page for instructions for setting up kafka with local pyspark: \
    `https://stackoverflow.com/questions/70374571/connecting-pyspark-with-kafka`
- Kafka version installed on my system: \
    `3.12:3.7.1`
- Bootstrap server hostname for kafka local server: \
    `localhost:9092`

### All the jar files that are downloaded below must be placed inside the `jars` folder of the spark installation directory in the localmachine:
-------------------------------------------------------------------------------------------------------------------------------------------------
- Required jar file for kafka, for this specific spark version `3.4.2`: \
    `spark-sql-kafka-0-10_2.12:3.4.2`
- Required links for setup:
- Maven central repository for `spark-sql-kafka-0-10_2.12:3.4.2`: \
    `https://central.sonatype.com/artifact/org.apache.spark/spark-sql-kafka-0-10_2.12/3.4.2`
- Download page for `spark-sql-kafka-0-10_2.12:3.4.2`: \
    `https://mvnrepository.com/artifact/org.apache.spark/spark-sql-kafka-0-10_2.12/3.4.2`
- Maven Central Repository for `kafka-clients:3.7.1`: \
    `https://central.sonatype.com/artifact/org.apache.kafka/kafka-clients`
- Download Page for `kafka-clients:3.7.1`: \
    `https://mvnrepository.com/artifact/org.apache.kafka/kafka-clients/3.7.1`

In [1]:
# from dotenv import load_dotenv
# load_dotenv()

True

In [1]:
import findspark
findspark.init()

In [2]:
scala_version = '2.12'
spark_version = '3.4.2'
kafka_client_version = '3.7.1' # kafka specific version installed in system, that is in "c:\kafka", client Jar file package of this version must be downloaded

packages = [
    f'org.apache.spark:spark-sql-kafka-0-10_{scala_version}:{spark_version}',
    f'org.apache.kafka:kafka-clients:{kafka_client_version}'
]

In [3]:
import time

In [4]:
from pyspark.sql import SparkSession

spark = ( 
    SparkSession
    .builder
    .master("local[*]")
    .appName("01-Intro to Spark Structured Streaming")
    .config("spark.jars.packages", ",".join(packages))
    .getOrCreate()
)
spark

In [6]:
# code for testing if hadoop is working fine with spark
# spark.range(100).write.format("parquet").save("./parquet_range")

In [5]:
kafka_topic_name = "first_topic"
kafka_broker = "localhost:9092"

In [6]:
streaming_df = spark.readStream\
    .format("kafka") \
    .option("kafka.bootstrap.servers", kafka_broker) \
    .option("subscribe", kafka_topic_name) \
    .option("startingOffsets", "earliest") \
    .load()

In [7]:
print( type(streaming_df) )

<class 'pyspark.sql.dataframe.DataFrame'>


In [9]:
streaming_df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [8]:
(
    streaming_df.writeStream
    .format("console")
    .outputMode("append")
    .option("checkpointLocation", "./checkpoints")
    # .trigger(availableNow=True)
    .start()
    # .awaitTermination()
)

In [11]:
cast_df = streaming_df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

In [12]:
(
    cast_df.writeStream
    .format("console")
    .outputMode("append")
    .option("checkpointLocation", "./checkpoints")
    # .trigger(availableNow=True)
    .start()
    # .awaitTermination()
)

In [19]:
query1 = streaming_df.writeStream.queryName("counting").format("memory").outputMode("append").start()
for x in range(5):
  spark.sql("select * from counting").show()
  time.sleep(5)

+---+-----+-----+---------+------+---------+-------------+
|key|value|topic|partition|offset|timestamp|timestampType|
+---+-----+-----+---------+------+---------+-------------+
+---+-----+-----+---------+------+---------+-------------+

+---+-----+-----+---------+------+---------+-------------+
|key|value|topic|partition|offset|timestamp|timestampType|
+---+-----+-----+---------+------+---------+-------------+
+---+-----+-----+---------+------+---------+-------------+

+---+-----+-----+---------+------+---------+-------------+
|key|value|topic|partition|offset|timestamp|timestampType|
+---+-----+-----+---------+------+---------+-------------+
+---+-----+-----+---------+------+---------+-------------+

+---+-----+-----+---------+------+---------+-------------+
|key|value|topic|partition|offset|timestamp|timestampType|
+---+-----+-----+---------+------+---------+-------------+
+---+-----+-----+---------+------+---------+-------------+

+---+-----+-----+---------+------+---------+--------

In [ ]:
st_query = (
    streaming_df.writeStream
    .format("")
    .trigger(availableNow=True)
    .start()
)